# From Chaos to Statistical Ensemble

## Elastic Collision between two masses
We start by looking into an elastic collision between two masses, $M_1$ and $M_2$.  
Two masses move on a periodic ring with positions in $[0, 1)$. This means we are modelling 1-dimensional elastic collisions.  
Initial velocities of the masses are $u_1$ and $u_2$, and the velocities after the collision are $v_1$, $v_2$.  
Final velocities are found through the conservation of momentum and the conservation of energy (with a neat trick of writing down the energy conservation as $M_1 (u_1^2 - v_1^2) = M_2 (v_2^2 - u_2^2)$, the conservation of momentum as $M_1(u_1 - v_1) = M_2(v_2 - u_2)$ and dividing the two equations) and the final result is 
$\begin{align}
    v_1 &= \frac{m1-m2}{m_1+m_2}u_1 + \frac{2m_2}{m_1+m_2}u_2 \\
    v_2 &= \frac{2m_1}{m_1+m_2}u_1 + \frac{m_2-m_1}{m_1+m_2}u_2
\end{align}$
The simulation is conducted in terms of 'events': the next collision is found and that is when the positions and velocities of the particles are updated (as opposed to numerical integrators, we would need them if there were some forces acting on the system).

In [1]:
# Importing the packaged that will be used throughout
import numpy as np
from decimal import Decimal

First step in creating the simulation was the creation of a simple Particle class that will store all the information about individual particles.

In [2]:
# Let's start with defining the classes that will be used in the simulation
# Base particle class
class Particle:
    """
    Base class for a particle. Not much is happening in here apart from storing
    the data for each particle and the update function which updates the data.
    """

    def __init__(self, mass, initPos, initVel, index=None):
        """"
        Arguments:
            mass (float) -> mass of the particle
            initPos (float) -> between 0 and 1, initial position of the particle
            initVel (float) -> pretty much any real number, initial velocity of the particle
            index (number) -> number assigned to this particle, defaults to None
        """
        self.mass = Decimal(str(mass))
        self.position = Decimal(str(initPos))
        self.velocity = Decimal(str(initVel))
        self.index = index

    def update(self, newPos=None, newVel=None):
        if newPos is not None:
            self.position = newPos

        if newVel is not None:
            self.velocity = newVel

Before moving to simulating the system, a discussion about auxiliary functions used throughout the code is waranted.

Important part of the simulation is finding the ring distance between any two particles. This is not as straightforward as it seems, as the distance we are interested in can be in the either side of the ring, depending on the relative velocities of the two particles. Velocity is calculated from the inertial frame of reference of particle 1 (first one to the 'left' of the other particle) and the calculation is split into two cases, depending on the relative velocity between the two particles.

In [3]:
# Auxiliary functions used in the code
def ring_distance(particle1, particle2):  # Assume that the particle1 is the first one to the left of the particle2
    pos1, vel1 = particle1.position, particle1.velocity
    pos2, vel2 = particle2.position, particle2.velocity

    # Transfer into the inertial reference frame of particle 1
    velDiff = vel2 - vel1

    if velDiff < 0:  # The right particle is moving towards the left one
        # Find the right (from particle 1) distance between the particles
        if pos2 > pos1:
            return abs(pos2 - pos1)
        elif pos2 < pos1: # e.g. x_1 = 0.9, x_2 = 0.2
            return Decimal("1") - abs(pos2 - pos1)
        else:
            return Decimal("1")
    elif velDiff > 0:
        # Find the left (from particle 1) distance between the particles
        if pos2 > pos1:
            return Decimal("1") - abs(pos2 - pos1)
        elif pos2 < pos1:
            return abs(pos2 - pos1)
        else:
            return Decimal("1")

Other important functions are: 

The usual elastic_collision function which simply finds after collision velocities of the two inputed particles.

More interesting one can_collide particles. This one is important because it checks whether the two particles that are being inspected can collide at all. The check is conducted through the difference in velocities, and the order of the indices. Throughout the simulation, indices have to stay in a particular order (Levi-Civita tensor on their ordering (from left of the ring to the right) must always be 1) and this is utilized in making sure the particles do not phase one through each other.

Finally there is n3_check_for_undetected_collision. In the case of a triple collisions (in a system of 3 particles) this ensures that all three particles are collided, in turns. It doesn't work on systems with more particles, though I think it should be easy to extend the function to arbitrary number of particles.

In [4]:
def elastic_collision(M1, u1, M2, u2):
    v1 = (M1 - M2)/(M1 + M2) * u1 + 2*M2/(M1 + M2) * u2
    v2 = 2*M1/(M1+M2) * u1 + (M2 - M1)/(M1 + M2) * u2
    return v1, v2


def can_collide(particlel, particler):
    vell, indexl = particlel.velocity, particlel.index
    velr, indexr = particler.velocity, particler.index
    veldif = velr - vell
    indexdif = indexr - indexl
    if veldif > 0 and indexdif < 0:
        return False
    elif veldif < 0 and indexdif < 0:
        return True
    elif veldif > 0 and indexdif > 0:
        return False
    elif veldif < 0 and indexdif > 0:
        return True


def n3_check_for_undetected_collision(collisionIndices): # n3 stands for this only works for three particles
    if len(collisionIndices) == 2:
        collisionIndices.append(collisionIndices[0])

We have come to a class that simulates the entire Sistem of particles on a ring.

Internally it stores the following information

* Number of particles
* Particle objects themselves in a list
* Particle positions in a list
* Particle velocities in a list
* Particle indices in a list (initially particles are indexed with integer indices starting from 0, from left to right 0-1)
* Total momentum of all particles
* Total energy of all particles
* Time until the next event happens
* Indices of particles that are involved in the next event
* Masses of the particles in a list

The simulation itself is event driven. Given a phase state, next collision to occur is found and particles are subsequently updated with the calculated data. Key step in the update is the index sorting, this means the particles are stored in a list such that index 0 particle is always the first element of the list. This ensures the two particles never 'phase' one through another and that their ordering is always constant. Collision detection is coded in such that only the collision with the particle on the 'left' (or with -1 index) is checked. This is sufficient to cover all possible collisions and minimize the amount of checks needed. 

In [5]:
# Class that simulates a sistem of particlese and their interactions.
class Sistem:
    """
    Class which simulates a system of particles.
    """

    def __init__(self, particleNum, masses='random', initPositions='random', initVelocities='random'):
        """Sistem class which contains all the particles in the ring and their interactions.

        Args:
            particleNum (integer): Number of particles on the ring.
            masses (list, optional): List of masses for the particles. Must be in given in increasing positions of the particles. Defaults to 'random'.
            initPositions (list, optional): List of initial positions of the particles. Must be in increasing order. Defaults to 'random'.
            initVelocities (str, optional): List of initial velocities of the particles. Must be in increasing positions of the particles. Defaults to 'random'.

        Raises:
            Exception: Check if the masses input is correct.
            Exception: Check if the len of the masses list is the same as the number of particles.
            Exception: Check if the initPositions input is correct.
            Exception: Check if the len of the initPositions is the same as the number of particles.
            Exception: Check if the initVelocities input is correct.
            Exception: Check if the len of the initVelocities is the same as the number of particles.
        """
        self.particleNum = particleNum

        if masses == 'random':
            masses = np.random.random(size=particleNum) + 0.00001  # Adding a small constant to avoid assigning a mass of 0
        elif not isinstance(masses, list) and not isinstance(masses, np.ndarray):
            raise Exception('Not a valid input for the masses, if not "random" the input has to be a list of masses!')
        elif len(masses) != particleNum:
            raise Exception('Differing amount of masses to the amount of particles was provided!')

        if initPositions == 'random':
            initPositions = np.sort(np.random.random(size=particleNum))  # Sort the array so that the nearest neighbors of a particle i are always particles (i+1) and (i-1)
        elif not isinstance(initPositions, list) and not isinstance(initPositions, np.ndarray):
            raise Exception('Not a valid input for the masses, if no "random" the input has to be a list of positions!')
        elif len(initPositions) != particleNum:
            raise Exception('Differing amount of positions to the amount of particles was provided!')

        if initVelocities == 'random':
            initVelocities = 2*np.random.random(size=particleNum) - 1  # Initialize random velocities in the range [-1, 1)
        elif not isinstance(initVelocities, list) and not isinstance(initVelocities, np.ndarray):
            raise Exception('Not a valid input for the velocities, if not "random" the input has to be a list of velocities!')
        elif len(initVelocities) != particleNum:
            raise Exception('Differing amount of velocities to the amount of particles was provided!')

        self.particles = [Particle(mass, initPos, initVel, index) for index, (mass, initPos, initVel) in enumerate(zip(masses, initPositions, initVelocities))]
        self.positions = [particle.position for particle in self.particles]
        self.velocities = [particle.velocity for particle in self.particles]
        self.indexes = [particle.index for particle in self.particles]
        self.momentum = sum([particle.mass * particle.velocity for particle in self.particles])
        self.energy = sum([Decimal("0.5") * particle.mass * particle.velocity**2 for particle in self.particles])
        time, collidingParticles = self.find_next_event_s()
        self.time = time  # Store the time interval for which the particles are in the current state
        self.collideIndices = collidingParticles  # Store the indices of the two particles that will collide
        self.masses = masses


    def find_next_event_s(self):
        """
        Function that finds when the next collision will happen, and what two particles will collide.
        Returns a tuple of the format (time to collision, (index of particle 1 that collides, index of particle 2 that collides))
        """
        shortestTime = 1e10  # Start with a big number
        collideParticlesIndex = []
        for i in range(self.particleNum):
            if can_collide(self.particles[i-1], self.particles[i]):
                distanceLeft = ring_distance(self.particles[i-1], self.particles[i])  # Only check with the particle to the left, to avoid doublechecking

                if self.particles[i-1].velocity == self.particles[i].velocity:  # if the particles have the same velocity they will never collide
                    time = 1e10
                else:
                    time = distanceLeft / (abs(self.particles[i-1].velocity - self.particles[i].velocity))

                if time < shortestTime:
                    shortestTime = time
                    collideParticlesIndex = [(i-1, i)]
                elif time == shortestTime:
                    collideParticlesIndex.append((i-1, i))

        if self.particleNum == 3:  # Since it only works for 3 particles
            n3_check_for_undetected_collision(collideParticlesIndex)
        return (shortestTime, collideParticlesIndex)


    def update_particles(self, time, collidingIndices):
        """
        Function updates the positions of all particles and velocities of particles that collided in the event.
        """
        for particle in self.particles:
            # Update the positions of all the particles, the ones colliding will be at the same position
            vel = particle.velocity
            pos = particle.position
            newPos = (pos + time*vel) % 1
            if newPos < 0:  # For some reason the % operation on decimal objects can return negative values
                newPos += Decimal("1")
            particle.update(newPos=newPos)
        

        for index in collidingIndices:
            left, right = index

            # Update the velocities of the particles that have collided
            mass1 = self.particles[left].mass
            vel1 = self.particles[left].velocity
            mass2 = self.particles[right].mass
            vel2 = self.particles[right].velocity

            newVel1, newVel2 = elastic_collision(mass1, vel1, mass2, vel2)

            self.particles[left].update(newVel=newVel1)
            self.particles[right].update(newVel=newVel2)

        # Recalculate the energy and momentum
        self.momentum = sum([particle.mass * particle.velocity for particle in self.particles])
        self.energy = sum([Decimal("0.5") * particle.mass * particle.velocity**2 for particle in self.particles])

        # Sort the particle list by their index
        self.particles.sort(key=lambda x: x.index)

        # Update the positions and velocities in the sistem class for purposes of logging them later
        self.positions = [particle.position for particle in self.particles]
        self.velocities = [particle.velocity for particle in self.particles]

        # Find the time the particles will stay in this state and the next two colliding particles
        time, colParIndex = self.find_next_event_s()
        self.time = time
        self.collideIndices = colParIndex

Finally, we have a Simulation wrapper class. This object controls the system inputs: number of collisions, number of particles, particle masses, initial positions and initial velocities.

The run method runs the simulation. There are additional parameters that have to be passes to be useful.

* shouldPrint parameter determines what sistem properties are printed out during the simulation. Available options are: 'positions', 'velocities', 'time', 'energy', 'momentum', 'indexes'.
* shouldLog parameter does the same thing as shouldPrint but writes down those values in a file
* filename determines the name of the file in which the data is stored

In [6]:
# Class wrapper for a single simulation experiment.
class Simulation:

    def __init__(self, collisionNumber, particleNumber, masses='random', initPoss='random', initVels='random'):
        """
        Simulation environment.

        Args:
            collisionNumber (int): Number of collisions to simulate.
            particleNumber ([type]): Number of particles to simulate.
            masses (list, optional): If a list is inputed make sure there is a mass for each particle. Defaults to 'random'.
            initPoss (list, optional): If a list is inputed make sure there is a position for each particle. Defaults to 'random'.
            initVels (list, optional): If a list is inputed make sure there is a velocity for each particle. Defaults to 'random'.
        """
        self.collisionNumber = collisionNumber
        self.sistem = Sistem(particleNum=particleNumber, masses=masses, initPositions=initPoss, initVelocities=initVels)


    def run(self, shouldPrint=None, shouldLog=None, filename=None):
        """
        This function collides the particles preset amount of times.

        Args:
            shouldPrint (list, optional): List of sistem object attributes you would like to have printed out. 
                                          Options are: 'positions', 'velocities', 'time', 'energy', 'momentum', 'indexes'.
                                          Defaults to None.
            shouldLog (list, optional): List of sistem object attributes you would like to have saved.
                                        Options are: 'positions', 'velocities', 'time', 'energy', 'momentum', 'indexes'.
                                        Defaults to None.
            filename (string, optional): Path and the filename of where you want the shouldLog data saved. If using shouldLog
                                        argument please provide the path as well.
        """
        if filename is not None:
            f = open(filename, 'w')

            # Write down number of particles and their masses
            massStr = ' '.join(str(mass) for mass in self.sistem.masses)

            f.write(f'Number of particles is: {self.sistem.particleNum} | Masses: {massStr} \n')
            
            attributeStr = '|'.join(attribute for attribute in shouldLog)
            f.write(attributeStr + '\n')

        for _ in range(self.collisionNumber):
            
            if shouldPrint is not None:
                for attribute in shouldPrint:  # Just fancy pritting tools, not important to the physics
                    if hasattr(self.sistem, attribute):
                        values = getattr(self.sistem, attribute)
                        if isinstance(values, list):
                            print(attribute + ':', end=' ')
                            for value in values:
                                if isinstance(value, tuple):
                                    print(' '.join(str(item) for item in value), end=' ')
                                else:
                                    print(f'{value:.2f}', end=' ')
                            print('', end='| ')
                        else:
                            print(f'{attribute}:{getattr(self.sistem, attribute):.3f} |', end=' ')

            if shouldPrint is not None:
                print('')

            if filename is not None:
                for attribute in shouldLog:
                    if hasattr(self.sistem, attribute):
                        values = getattr(self.sistem, attribute)
                        if isinstance(values, list):
                            if isinstance(values[0], tuple):
                                for value in values:
                                    f.write(' '.join(str(item) for item in value))
                            else:
                                f.write(' '.join(str(value) for value in values) + '|')
                        else:
                            f.write(str(getattr(self.sistem, attribute)) + '|')
                f.write('\n')

            self.sistem.update_particles(self.sistem.time, self.sistem.collideIndices)

        if filename is not None:
            f.close()

In [7]:
# Example usage
experiment = Simulation(collisionNumber=10, particleNumber=3,
                        masses=[1, 1, 1], initPoss=[0.2, 0.5, 0.8],
                        initVels=[1, 0, -1])
experiment.run(shouldPrint=['positions', 'velocities'])

positions: 0.20 0.50 0.80 | velocities: 1.00 0.00 -1.00 | 
positions: 0.50 0.50 0.50 | velocities: -1.00 0.00 1.00 | 
positions: 0.00 0.50 0.00 | velocities: 1.00 0.00 -1.00 | 
positions: 0.50 0.50 0.50 | velocities: -1.00 0.00 1.00 | 
positions: 0.00 0.50 0.00 | velocities: 1.00 0.00 -1.00 | 
positions: 0.50 0.50 0.50 | velocities: -1.00 0.00 1.00 | 
positions: 0.00 0.50 0.00 | velocities: 1.00 0.00 -1.00 | 
positions: 0.50 0.50 0.50 | velocities: -1.00 0.00 1.00 | 
positions: 0.00 0.50 0.00 | velocities: 1.00 0.00 -1.00 | 
positions: 0.50 0.50 0.50 | velocities: -1.00 0.00 1.00 | 


Having the data stored any subsequent analysis can be conducted reading the data in and analysing it.